In [1]:
from qmcpy import *
import numpy as np
from pandas import *
import scipy as sc
from sympy import *
import matplotlib.pyplot as plt
from scipy.special import erf
import math

In [16]:
class RQMCEstimator:
    def __init__(self, n=4096, h=0.001,k=lambda x:1/np.sqrt(2*np.pi)*np.exp(-x**2/2),sequence='Sobol',function='ECO'):
        if function=='Gaussian':
            self.g=lambda x:1/np.sqrt(2*np.pi)*np.exp(-x**2/2)
        elif function=='ECO': 
            T=1
            K=30
            r=0.1
            sigma=0.2
            self.g=lambda x:(x * (1.0 + np.vectorize(math.erf)(((np.log(x / K) + 
                (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))) / np.sqrt(2.0))) / 2.0 - 
                K * np.exp(-r * T) * (1.0 + np.vectorize(math.erf)(((np.log(x / K) + 
                (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T)) - sigma * np.sqrt(T)) / np.sqrt(2.0))) / 2.0)
        elif function=='EPO':
            T=1
            K=30
            r=0.1
            sigma=0.2
            self.g=lambda x:(-x * (1.0 + np.vectorize(math.erf)(-((np.log(x / K) + 
                (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))) / np.sqrt(2.0))) / 2.0 + 
                K * np.exp(-r * T) * (1.0 + np.vectorize(math.erf)(-((np.log(x / K) + 
                (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T)) - sigma * np.sqrt(T)) / np.sqrt(2.0))) / 2.0)
        else:
            raise ValueError("Invalid input. Must be Gaussian, ECO or EPO")
        if sequence=='Sobol':
            self.sampler=Sobol(1).gen_samples(n)
            self.a=min(self.g(self.sampler))
            self.b=max(self.g(self.sampler))
        elif sequence=='Lattice':
            self.sampler=Lattice(1).gen_samples(n)
            self.a=min(self.g(self.sampler))
            self.b=max(self.g(self.sampler))
        elif sequence=='Normal':
            self.sampler=np.random.normal(size=n)
            self.a=min(self.sampler)
            self.b=max(self.sampler)
        else:
            raise ValueError("Invalid input. Must be Sobol, Lattice or Normal")
        self.n=n 
        self.h=h
        self.k=k
        print("Lower bound of estimation interval (a) is:",self.a)
        print("Upper bound of estimation interval (b) is:",self.b)

    def density_estimation(self,x):
        if sequence=='Sobol'or sequence=='Lattice':
            f=zeros(len(x),1)
            g=self.g(self.sampler)
            for i in range (len(x)):
                for p in range (self.n):
                    f[i]+=(1/self.n)*(1/self.h)*self.k(((x[i]-g[p])/self.h))
        else:
            f=zeros(len(x),1)
            g=self.sampler
            for i in range (len(x)):
                for p in range (self.n):
                    f[i]+=(1/self.n)*(1/self.h)*self.k(((x[i]-g[p])/self.h))
        return f        
    
    def mise(self):
        est=np.squeeze(self.density_estimation())
        #real_distrib=self.gaussian(self.points,2, 1) we define a distribution to estimate
        integrand=((est-real_distrib)*np.random.normal(2,1,self.m))**2
        ex=np.trapz(integrand)
        return ex
        return mise


In [17]:
est=RQMCEstimator()
x=[0.35]
f=est.density_estimation(x)
print(f)

[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
Lower bound of estimation interval (a) is: [0.]
Upper bound of estimation interval (b) is: [0.]
Matrix([[1.00003851780413]])


In [10]:
class RQMCEstimator:
    def __init__(self, n=4096, h=0.001,k=lambda x:1/np.sqrt(2*np.pi)*np.exp(-x**2/2),sequence='Sobol'):
        self.g=lambda x:1/np.sqrt(2*np.pi)*np.exp(-x**2/2)
        if sequence=='Sobol':
            self.sampler=Sobol(1).gen_samples(n)
            self.a=min(self.g(self.sampler))
            self.b=max(self.g(self.sampler))
        elif sequence=='Lattice':
            self.sampler=Lattice(1).gen_samples(n)
            self.a=min(self.g(self.sampler))
            self.b=max(self.g(self.sampler))
        elif sequence=='Normal':
            self.sampler=np.random.normal(size=n)
            self.a=min(self.sampler)
            self.b=max(self.sampler)
        else:
            raise ValueError("Invalid input. Must be Sobol, Lattice or Normal")
        self.n=n 
        self.h=h
        self.k=k
        print("Lower bound of estimation interval (a) is:",self.a)
        print("Upper bound of estimation interval (b) is:",self.b)

    def density_estimation(self,x):
        if sequence=='Sobol'or sequence=='Lattice':
            f=zeros(len(x),1)
            g=self.g(self.sampler)
            for i in range (len(x)):
                for p in range (self.n):
                    f[i]+=(1/self.n)*(1/self.h)*self.k(((x[i]-g[p])/self.h))
        else:
            f=zeros(len(x),1)
            g=self.sampler
            for i in range (len(x)):
                for p in range (self.n):
                    f[i]+=(1/self.n)*(1/self.h)*self.k(((x[i]-g[p])/self.h))
        return f        
    
    def mise(self):
        est=np.squeeze(self.density_estimation())
        #real_distrib=self.gaussian(self.points,2, 1) we define a distribution to estimate
        integrand=((est-real_distrib)*np.random.normal(2,1,self.m))**2
        ex=np.trapz(integrand)
        return ex
        return mise
    
    

    
    
class OptionEstimator(RQMCEstimator):
    def __init__(self, n=4096, h=0.001,k=lambda x:1/np.sqrt(2*np.pi)*np.exp(-x**2/2),sequence='Sobol',option='ECO'):
        super().__init__(n, h,k,sequence)
        if option=='ECO': 
            T=1
            K=30
            r=0.1
            sigma=0.2
            self.g=lambda x:(x * (1.0 + np.vectorize(math.erf)(((np.log(x / K) + 
                (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))) / np.sqrt(2.0))) / 2.0 - 
                K * np.exp(-r * T) * (1.0 + np.vectorize(math.erf)(((np.log(x / K) + 
                (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T)) - sigma * np.sqrt(T)) / np.sqrt(2.0))) / 2.0)
        elif option=='EPO':
            T=1
            K=30
            r=0.1
            sigma=0.2
            self.g=lambda x:(-x * (1.0 + np.vectorize(math.erf)(-((np.log(x / K) + 
                (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))) / np.sqrt(2.0))) / 2.0 + 
                K * np.exp(-r * T) * (1.0 + np.vectorize(math.erf)(-((np.log(x / K) + 
                (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T)) - sigma * np.sqrt(T)) / np.sqrt(2.0))) / 2.0)
        else:
            raise ValueError("Invalid input. Must be ECO or EPO")

    def density_estimation(self,x):
        return super().density_estimation(x)   
    def mise(self):
        est=np.squeeze(self.density_estimation())
        #real_distrib=self.gaussian(self.points,2, 1) we define a distribution to estimate
        integrand=((est-real_distrib)*np.random.normal(2,1,self.m))**2
        ex=np.trapz(integrand)
        return ex
        return mise



In [11]:
est=OptionEstimator()
x=[0.35]
f=est.density_estimation(x)
print(f)

Lower bound of estimation interval (a) is: [0.24199413]
Upper bound of estimation interval (b) is: [0.39894228]
Matrix([[0.986298622280639]])
